In [2]:
import boto3

bucket_name_str = "erl-2021-07-17-s3site"

client = boto3.client('apigateway', region_name='us-east-2')
client_bucket = boto3.client('s3', region_name='us-east-2')
bucket = client_bucket.list_buckets()

response = client.get_rest_apis(
    limit=10
)
print(response['items'])

[{'id': 'cyeac92bt6', 'name': 'Fancy-Api', 'description': 'test api', 'createdDate': datetime.datetime(2021, 7, 27, 21, 34, 15, tzinfo=tzlocal()), 'minimumCompressionSize': 123, 'apiKeySource': 'HEADER', 'endpointConfiguration': {'types': ['REGIONAL']}, 'disableExecuteApiEndpoint': False}, {'id': 'sm18ikuwp4', 'name': 'dragons', 'description': 'get dragons', 'createdDate': datetime.datetime(2021, 7, 21, 3, 58, 43, tzinfo=tzlocal()), 'apiKeySource': 'HEADER', 'endpointConfiguration': {'types': ['REGIONAL']}, 'disableExecuteApiEndpoint': False}]


In [3]:
ID = response['items'][0]['id']
resources = client.get_resources(restApiId=ID)
print(resources)

{'ResponseMetadata': {'RequestId': 'ed23fd34-73dd-40df-ae88-02f7343a4ac3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 28 Jul 2021 03:17:25 GMT', 'content-type': 'application/json', 'content-length': '409', 'connection': 'keep-alive', 'x-amzn-requestid': 'ed23fd34-73dd-40df-ae88-02f7343a4ac3', 'x-amz-apigw-id': 'DKTa5LxlCYcEdyg='}, 'RetryAttempts': 0}, 'items': [{'id': 'ad80p7', 'parentId': 'prkz7qtz7i', 'pathPart': 'get_reviews', 'path': '/get_reviews', 'resourceMethods': {'GET': {}}}, {'id': 'prkz7qtz7i', 'path': '/'}, {'id': 'tymi9k', 'parentId': 'prkz7qtz7i', 'pathPart': 'create_report', 'path': '/create_report', 'resourceMethods': {'POST': {}}}, {'id': 'um97x0', 'parentId': 'prkz7qtz7i', 'pathPart': 'get_av_star_rating', 'path': '/get_av_star_rating', 'resourceMethods': {'GET': {}}}]}


In [4]:
create_report_id = [resource for resource in resources["items"] if resource["path"] == "/create_report"][0]["id"]
get_reviews_id = [resource for resource in resources["items"] if resource["path"] == "/get_reviews"][0]["id"]
get_av_star_rating_id = [resource for resource in resources["items"] if resource["path"] == "/get_av_star_rating"][0]["id"]

In [5]:
options_method = client.put_method(
    restApiId=ID,
    resourceId=create_report_id,
    httpMethod='OPTIONS',
    authorizationType='NONE'
)

In [6]:
options_mock = client.put_integration(
    restApiId=ID,
    resourceId=create_report_id,
    httpMethod='OPTIONS',
    type='MOCK',
    requestTemplates={
        'application/json': '{"statusCode": 200}'
    }
)

In [7]:
options_method_response = client.put_method_response(
    restApiId=ID,
    resourceId=create_report_id,
    httpMethod='OPTIONS',
    statusCode='200',
    responseParameters={
        'method.response.header.Access-Control-Allow-Headers': True,
        'method.response.header.Access-Control-Allow-Origin': True,
        'method.response.header.Access-Control-Allow-Methods': True,
        'method.response.header.Access-Control-Allow-Credentials': True
    },
    responseModels={
        'application/json': 'Empty'
    }
)

In [8]:
options_integration_response = client.put_integration_response(
    restApiId=ID,
    resourceId=create_report_id,
    httpMethod='OPTIONS',
    statusCode='200',
    responseParameters={
        'method.response.header.Access-Control-Allow-Headers': '\'Content-Type,X-Amz-Date,Authorization,X-Api-Key,X-Amz-Security-Token\'',
        'method.response.header.Access-Control-Allow-Methods': '\'POST,OPTIONS\'',
        'method.response.header.Access-Control-Allow-Origin': "'https://" + bucket_name_str + ".s3-us-west-2.amazonaws.com'",
        'method.response.header.Access-Control-Allow-Credentials': "'true'"

    },
    responseTemplates={
        'application/json': ''
    }
)